In [1]:
%load_ext autoreload
%autoreload 2
import mycode.vap as vap

In [2]:
# the goal is to check if document deletion by selection on fields that are not an attribute is possible

In [21]:
from vespa.package import (ApplicationPackage, Field, Schema, Document, Validation)
ap = ApplicationPackage(
    validations=[Validation(validation_id='indexing-change', until='2026-01-10')],
    name="hex",
    schema=[
        Schema(
            name="hex",
            document=Document(
                fields=[
                    Field(
                        name="f1",
                        type="int",
                        indexing=["attribute", "summary"],
                    ),
                    Field(
                        name="f2",
                        type="int",
                        indexing=[],
                    )
                ]
            )
        )
    ]
)

In [22]:
from vespa.deployment import VespaDocker

# In case running colima on macos run the following
# !sudo ln -sf $HOME/.colima/default/docker.sock /var/run/docker.sock
vespa_docker = VespaDocker(
    container_image="vespaengine/vespa:8.625.17",
)
vespa_client = vespa_docker.deploy(application_package=ap)

Waiting for configuration server, 0/60 seconds...
Waiting for configuration server, 5/60 seconds...
Waiting for application to come up, 0/300 seconds.
Waiting for application to come up, 5/300 seconds.
Waiting for application to come up, 10/300 seconds.
Application is up!
Finished deployment.


In [39]:
# Feed the same float number to both fields
vespa_client.feed_iterable([
    {
        'id': '1',
        'fields': {
            'f1': 1,
            'f2': 1,
        }
    },
    {
        'id': '2',
        'fields': {
            'f1': 2,
            'f2': 2,
        }
    }
], schema="hex", namespace="hex", callback=vap.feed_callback)

In [26]:
vespa_docker.restart_services()

config-sentinel was running with pid 160, sending SIGTERM
Waiting for exit (up to 15 minutes)
. DONE
configproxy was running with pid 93, sending SIGTERM
Waiting for exit (up to 15 minutes)
. DONE

Executing /opt/vespa/libexec/vespa/stop-configserver
configserver was running with pid 25, sending SIGTERM
Waiting for exit (up to 15 minutes)
. DONE

Waiting for configuration server.
Waiting for configuration server.
Starting configserver on 'hex'
runserver(configserver) running with pid: 1872

Waiting for application to come up, 0/120 seconds.
Waiting for application to come up, 5/120 seconds.
Application is up!
Starting config proxy using tcp/localhost:19070 as config source(s)
Waiting for config proxy to start on 'hex'
runserver(configproxy) running with pid: 2152
config proxy started after 0s (runserver pid 2152)
Starting services for VESPA_HOSTNAME='hex'
runserver(config-sentinel) running with pid: 2214



In [27]:
vespa_client.get_data(data_id='1', schema="hex", namespace="hex").json

{'pathId': '/document/v1/hex/hex/docid/1',
 'id': 'id:hex:hex::1',
 'fields': {'f2': 1, 'f1': 1}}

In [28]:
vespa_client.get_data(data_id='2', schema="hex", namespace="hex").json

{'pathId': '/document/v1/hex/hex/docid/2',
 'id': 'id:hex:hex::2',
 'fields': {'f2': 2, 'f1': 2}}

In [40]:
visited = vespa_client.visit(content_cluster_name="hex_content", schema="hex", namespace="hex")
for slice in visited:
    for doc in slice:
        print(doc.json)

{'pathId': '/document/v1/hex/hex/docid/', 'documents': [{'id': 'id:hex:hex::2', 'fields': {'f2': 2, 'f1': 2}}, {'id': 'id:hex:hex::1', 'fields': {'f2': 1, 'f1': 1}}], 'documentCount': 2}


In [30]:
visited = vespa_client.visit(content_cluster_name="hex_content", schema="hex", namespace="hex", selection='hex.f1==2')
for slice in visited:
    for doc in slice:
        print(doc.json)

{'pathId': '/document/v1/hex/hex/docid/', 'documents': [{'id': 'id:hex:hex::2', 'fields': {'f2': 2, 'f1': 2}}], 'documentCount': 1}


In [33]:
visited = vespa_client.visit(content_cluster_name="hex_content", schema="hex", namespace="hex", selection='hex.f2==1')
for slice in visited:
    for doc in slice:
        print(doc.json)

{'pathId': '/document/v1/hex/hex/docid/', 'documents': [{'id': 'id:hex:hex::1', 'fields': {'f2': 1, 'f1': 1}}], 'documentCount': 1}


In [38]:
vespa_client.delete_data(namespace="hex", schema="hex", data_id='', selection='true', cluster='hex_content').json

{'pathId': '/document/v1/hex/hex/docid/', 'documentCount': 0}

In [41]:
vespa_client.delete_data(namespace="hex", schema="hex", data_id='', selection='hex.f1==2', cluster='hex_content').json

{'pathId': '/document/v1/hex/hex/docid/', 'documentCount': 1}

In [42]:
visited = vespa_client.visit(content_cluster_name="hex_content", schema="hex", namespace="hex")
for slice in visited:
    for doc in slice:
        print(doc.json)

{'pathId': '/document/v1/hex/hex/docid/', 'documents': [{'id': 'id:hex:hex::1', 'fields': {'f2': 1, 'f1': 1}}], 'documentCount': 1}


In [43]:
vespa_client.delete_data(namespace="hex", schema="hex", data_id='', selection='hex.f2==1', cluster='hex_content').json

{'pathId': '/document/v1/hex/hex/docid/', 'documentCount': 1}

In [44]:
visited = vespa_client.visit(content_cluster_name="hex_content", schema="hex", namespace="hex")
for slice in visited:
    for doc in slice:
        print(doc.json)

{'pathId': '/document/v1/hex/hex/docid/', 'documents': [], 'documentCount': 0}
